In [62]:
import os
from joblib import dump, load
import pickle
import sys
sys.path.insert(0, os.path.join(os.getcwd(),os.pardir))
from collections import Counter
import numpy as np

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

In [63]:
svc = load("best_linearSVC.joblib")

In [8]:
FILES  = ["train","val","test"]
FOLDER = os.path.join(os.getcwd(),os.pardir,"recipe_1m_analysis","data")
file = FILES[0]

In [49]:
enc = load("scikit_vocab_cuisine.joblib")

In [13]:
with open(os.path.join(FOLDER,"recipe1m_"+file+".pkl"),"rb") as f:
    data = pickle.load(f)

In [34]:
len(data)

268012

In [25]:
ingr_input = [";".join([ingr.name for ingr in recipe["ingredients"]]) for recipe in data.values() ]

In [17]:
vocab = load("scikit_vocab.joblib")

In [35]:
len(vocab)

70554

In [27]:
cv = CountVectorizer(vocabulary=vocab)
X = cv.fit_transform(ingr_input)

In [66]:
prediction = svc.predict(X)
len(prediction)

268012

In [55]:
str_pred =enc.inverse_transform(prediction)

In [56]:
Counter(str_pred)

Counter({'NorthAmerican': 227777,
         'SouthernEuropean': 17596,
         'WesternEuropean': 6502,
         'Asian': 3855,
         'LatinAmerican': 9505,
         'EastAsian': 1259,
         'SouthAsian': 1376,
         'SoutheastAsian': 89,
         'MiddleEastern': 20,
         'EasternEuropean': 8,
         'NorthernEuropean': 24,
         'African': 1})

In [69]:
len(str_pred)

268012

In [59]:
for i,v in enumerate(data.values()):
    v["cuisine"]=str_pred[i]
    
    if i<100:
        print(v["title"],v["cuisine"])

['dilly', 'macaroni', 'salad', 'recipe'] NorthAmerican
['gazpacho'] NorthAmerican
['cool', "'", 'n', 'easy', 'creamy', 'watermelon', 'pie'] NorthAmerican
['kombu', 'tea', 'grilled', 'chicken', 'thigh'] NorthAmerican
['pink', 'sangria'] NorthAmerican
['pineapple', 'loaf'] NorthAmerican
['tinklee', "'s", 'vanilla', 'crack'] NorthAmerican
['erin', "'s", 'mashed', 'potatoes'] NorthAmerican
['banana', 'cream', 'pie', 'smoothie'] NorthAmerican
['frosty', 'lemon', 'ice', 'cream', 'sandwiches'] NorthAmerican
['apple', 'cinnamon', 'french', 'toast', 'strata'] NorthAmerican
['margarita', 'madras'] NorthAmerican
['lobster', 'on', 'the', 'barbie'] NorthAmerican
['penne', 'with', 'smoked', 'trout', 'and', 'sugar', 'snap', 'peas'] NorthAmerican
['cheesy', 'herbed', 'egg', 'sandwich'] NorthAmerican
['balsamic', 'chicken', 'pasta', 'with', 'fresh', 'cheese'] SouthernEuropean
['chirashi', '(', 'scattered', ')', 'sushi'] NorthAmerican
['hot', 'corned', 'beef', 'buns'] NorthAmerican
['creme', 'curd', 'cu

In [70]:
#95% pred
threshold=0.95
y_pred_prob = svc.predict_proba(X)
max_prob = y_pred_prob.max(axis=1)
y_pred = y_pred_prob.argmax(axis=1)

to_del = np.argwhere(max_prob < threshold).T[0]

partial_y_pred = np.delete(y_pred,to_del)
partial_y_pred

array([6, 6, 6, ..., 6, 6, 6], dtype=int64)

In [71]:
len(partial_y_pred)

7250

In [72]:
Counter(partial_y_pred)

Counter({6: 7032, 4: 117, 10: 89, 8: 4, 1: 6, 11: 2})